In [ ]:
import os 
import io
import openai
from transformers.models.imagegpt.modeling_imagegpt import IMAGEGPT_INPUTS_DOCSTRING
from transformers import GPT2Tokenizer
import pandas as pd 
from pandas.io import json
from numpy import nan
import time
import csv
import shutil
import datetime
import pytz
import re
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
import random
import itertools
import random
import multiprocessing
import time
import os
import time
import random

In [ ]:
multiprocessing.cpu_count()

In [ ]:
openai.api_key = "OPENAI_API_KEY"
section_no = "PROTEIN_Prompts"

In [ ]:
#Generate All the variations 
def get_combinations(prompt_paths, model_engines, datasets, folds_LLL, folds_HPRD50, folds_IEPA):
    combinations = []
    # Create dictionary for dataset-specific folds
    dataset_folds = {
        'LLL': folds_LLL,
        'HPRD50': folds_HPRD50,
        'IEPA': folds_IEPA
    }
    # Iterate over each dataset to get the dataset-specific folds and prompts
    for dataset in datasets:
        current_prompts = prompt_paths
        current_folds = dataset_folds[dataset]
        for combination in itertools.product(current_prompts, model_engines, [dataset], current_folds):
            combinations.append(combination)
    random.shuffle(combinations)
    return combinations

In [ ]:
def user_input(file_path):
    try:
        with open(file_path, 'r') as file:
            query = file.read()
            file_name = file_path.split('/')[-1]  # Extract the file name from the path
    except FileNotFoundError:
        print(f"{file_path} not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return query

In [ ]:
import os
def output_path(Run_no, dataset, temperature, prompt_no, model_engine):
    base = "Output/" + model_engine + "_OneSentence_"+section_no+"_" + str(temperature) + "/" + str(prompt_no) +"/"+dataset + "/"
    extension_path = dataset + "_T" + str(temperature) + "_" + prompt_no + "_Run" + str(Run_no)+'/'
    Implementation_base_path_output = os.path.join(base, extension_path)
    os.makedirs(Implementation_base_path_output, exist_ok=True)
    return Implementation_base_path_output

In [ ]:
def count_input_tokens(folder_path):
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  file_names = os.listdir(folder_path)
  max_num_input_lines = 0

  # Loop through each file and calculate number of tokens
  for file_name in file_names:
      # Check if file is a text file
      if file_name.endswith(".txt"):
          # Read file contents
          with open(os.path.join(folder_path, file_name), "r" , encoding='utf-8') as f:
              file_contents = f.read()
          with open(os.path.join(folder_path, file_name), "r" , encoding='utf-8') as fp:
              num_input_line = len(fp.readlines())
              print("num_input_line:",num_input_line)
          # Calculate number of tokens
          num_tokens = len(tokenizer.encode(file_contents))
          if (num_input_line>max_num_input_lines):
            max_num_input_lines = num_input_line
          print(f"{file_name}: {num_tokens} tokens")
    
  total_max_tokens = 10

  # Loop through each file and calculate number of tokens
  for file_name in file_names:
      # Check if file is a text file
      if file_name.endswith(".txt"):
          # Read file contents
          with open(os.path.join(folder_path, file_name), "r" , encoding='utf-8') as f:
              file_contents = f.read()
          # Calculate number of tokens
          num_tokens = len(tokenizer.encode(file_contents))
          total = num_tokens+ total_max_tokens
          print(f"{file_name}: Input: {num_tokens} tokens, Output: {total_max_tokens} : Total: {total}")
  print(total_max_tokens)
  return total_max_tokens

In [ ]:
#ChatGPT Completion
def get_completion(BACKOFF_OCCURRED_, model, query, Sentences, max_tokens,  temperature):
    prompt = f"""
        {query}
        {Sentences}
        """
    messages = [{"role": "user", "content": prompt}]
    tries = 0
    while True:
        try:
            start_time = time.time()
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            call_time = time.time()
            time_f = call_time - start_time
            break
        except (openai.error.RateLimitError, openai.error.ServiceUnavailableError, openai.error.APIError) as e:
            tries += 1
            max_backoff = 60  # Example: maximum of 60 seconds
            backoff_time =  min(5 + 5*(tries ** 2), max_backoff)
            with BACKOFF_OCCURRED_.get_lock():
                BACKOFF_OCCURRED_.value = True
            time.sleep(backoff_time)
            with BACKOFF_OCCURRED_.get_lock():
                BACKOFF_OCCURRED_.value = False
            print("Backoff Released\n")
    
    message = response['choices'][0]['message']['content']
    output_token = response['usage']['completion_tokens']
    input_token = response['usage']['prompt_tokens']
    now_utc = datetime.datetime.now(pytz.utc)
    timezone = pytz.timezone("US/Central")
    now_eastern = now_utc.astimezone(timezone)
    time_stamp = str(now_eastern)
    return message, input_token, output_token, time_f, time_stamp, tries

In [ ]:
def call_ChatGPT(BACKOFF_OCCURRED_, current_folds, Implementation_base_path_input, Implementation_base_path_output, Run, total_max_tokens, temperature, query, dataset, model):
    RETRY_COUNT = 5
    input_file_path = os.path.join(Implementation_base_path_input, f'{current_folds}')
    output_file_path = os.path.join(Implementation_base_path_output, f'{Run}_{current_folds}')
    time_track_path = os.path.join(Implementation_base_path_output, f'{temperature}_time_track.csv')

    while BACKOFF_OCCURRED_.value:
        print("\nBackoff occurred! Pausing all threads for a set duration...")
        sleep_time = random.randint(1, 5)
        time.sleep(sleep_time)

    with open(input_file_path) as f:
        lines = f.readlines()

    # Write header to the output file
    with open(output_file_path, "a") as f:
        print("Sentence ID,PPI", file=f)

    for line_id, line in enumerate(lines, start=1):

        line = line.strip()
        if line.startswith('Sentence ID'):
            continue 

        sentence_id, sentence_rest = line.split('\t', 1)

        for attempt in range(RETRY_COUNT):
            try:
                message, input_token, output_token, time_f, time_stamp, tries = get_completion(BACKOFF_OCCURRED_, model, query, Sentences=sentence_rest, max_tokens=total_max_tokens, temperature=temperature)

                print(f"Sentence ID={sentence_id}, Fold={current_folds}, Run={Run}, Temperature={temperature}.")

                with open(time_track_path, "a") as f:
                    print(dataset, ',', Run, ',', current_folds, ',', sentence_id, ',', temperature, ',', input_token, ',', output_token, ',', time_f, ',', time_stamp, ',', tries, file=f)

                with open(output_file_path, "a") as f:
                    print(f"{sentence_id},{message}", file=f)
                break
                
            except Exception as e:
                print(f"Error occurred: {e}. Retrying {attempt+1}/{RETRY_COUNT} for Sentence ID={sentence_id}.")
                time.sleep(1) 

            if attempt == RETRY_COUNT - 1:
                print(f"All {RETRY_COUNT} retries failed for Sentence ID {sentence_id} at Fold {current_folds}, Run {Run}.")
        if line_id % 10 == 0:
            print("Pausing for a short break...")
            time.sleep(1)
            

In [ ]:
BACKOFF_OCCURRED_ = multiprocessing.Value('b', False)  # 'b' denotes a boolean

def execute_code(run_no, prompt_no, model_engine, dataset, current_folds):
    global BACKOFF_OCCURRED_
    prompt_path = f"Prompts/PROTEIN_Prompts/{prompt_no}.txt"
    temperature = 0.0 
    Implementation_base_path_input = 'Datasets/PROTEIN_DATA/' + dataset + '/PROTEIN_all/input/'
    Implementation_base_path_output = output_path(run_no, dataset, temperature, prompt_no, model_engine)
    total_max_tokens = count_input_tokens(Implementation_base_path_input)
    call_ChatGPT(BACKOFF_OCCURRED_, current_folds, Implementation_base_path_input, Implementation_base_path_output, run_no, total_max_tokens, temperature, query, dataset,model = model_engine)
    
if __name__ == '__main__':
    prompt_no = ['Prompt15_OneSentence']
    model_engines = ["gpt-3.5-turbo-0613", "gpt-4-0613"]
    datasets = ["LLL", "IEPA", "HPRD50"]
    
    # List for LLL, IEPA and HPRD50 with 10 folds
    folds_LLL = [f"fold{i}.txt" for i in range(1, 11)]
    folds_HPRD50 = [f"fold{i}.txt" for i in range(1, 11)]
    folds_IEPA = [f"fold{i}.txt" for i in range(1, 11)]

    for Run_no in range(2, 11):
        all_combinations = get_combinations(prompt_no, model_engines, datasets, folds_LLL, folds_HPRD50, folds_IEPA)
        print(f"Total number of combinations: {len(all_combinations)}")
        args = [(Run_no,) + combo for combo in all_combinations]
        no_of_workers = 5
        with multiprocessing.Pool(no_of_workers) as pool:
            pool.starmap(execute_code, args)
            time.sleep(1)